In [1]:
import pandas as pd
import numpy as np 
from scipy.interpolate import SmoothBivariateSpline
import matplotlib.pyplot as plt

In [2]:
data = pd.read_excel('data/before_pre/data_adj.xlsx') # Took the laptop about 5.5 minutes

In [3]:
print(data.head())

   Unnamed: 0       date               symbol     exdate   last_date cp_flag  \
0          20 2012-01-03  SPXW 120106C1200000 2012-01-06  03/01/2012       C   
1          22 2012-01-03  SPXW 120106C1210000 2012-01-06  03/01/2012       C   
2          24 2012-01-03  SPXW 120106C1220000 2012-01-06  03/01/2012       C   
3          26 2012-01-03  SPXW 120106C1230000 2012-01-06  03/01/2012       C   
4          27 2012-01-03  SPXW 120106C1235000 2012-01-06  03/01/2012       C   

   strike_price  best_bid  best_offer  volume  ...  maturity         Open  \
0          1200      75.8        78.7       2  ...         3  1258.859985   
1          1210      66.0        68.8       1  ...         3  1258.859985   
2          1220      56.1        58.8       1  ...         3  1258.859985   
3          1230      46.2        49.1       1  ...         3  1258.859985   
4          1235      41.4        44.1       1  ...         3  1258.859985   

          High          Low        Close    Adj Close   

In [4]:
data = data.drop(columns="Unnamed: 0")

In [5]:
print(data.iloc[:int(0.7*len(data))].tail())
print(data.iloc[int(0.7*len(data)):int(0.8*len(data))].tail())
print(data.iloc[int(0.8*len(data)):].head())
# Train set includes observations untill 06-12-2021
# Validation set includes observations utill 19-05-2022
# Test set includes observations from 20-05-2022 onwards
# smooth only the train set

             date               symbol     exdate   last_date cp_flag  \
579620 2021-12-06  SPXW 211210P3850000 2021-12-10  06/12/2021       P   
579621 2021-12-06  SPXW 211210P3860000 2021-12-10  06/12/2021       P   
579622 2021-12-06  SPXW 211210P3870000 2021-12-10  06/12/2021       P   
579623 2021-12-06  SPXW 211210P3875000 2021-12-10  06/12/2021       P   
579624 2021-12-06  SPXW 211210P3880000 2021-12-10  06/12/2021       P   

        strike_price  best_bid  best_offer  volume  open_interest  ...  \
579620          3850      0.85        1.00    3770           5939  ...   
579621          3860      0.90        1.00       2             85  ...   
579622          3870      0.90        1.00       2             70  ...   
579623          3875      0.95        1.05      55            359  ...   
579624          3880      0.95        1.05       7            139  ...   

        maturity         Open         High          Low        Close  \
579620         4  4548.370117  4612.600098  

In [6]:
for i in data['date'].unique():
    subset = data[data['date']==i][['maturity', 'moneyness']]
    uniq_mat = subset['maturity'].unique()
    mon_range = subset['moneyness']

    print(i, uniq_mat, np.min(mon_range), np.max(mon_range), len(subset))

2012-01-03T00:00:00.000000000 [3] 0.9565992951310861 1.06866950543933 49
2012-01-04T00:00:00.000000000 [2] 0.9750382053435114 1.073361385714286 35
2012-01-05T00:00:00.000000000 [1] 0.956014969402985 1.041512243089431 28
2012-01-06T00:00:00.000000000 [5] 0.9500446535315984 1.11599131790393 60
2012-01-09T00:00:00.000000000 [4] 0.966566000754717 1.080759452320675 43
2012-01-10T00:00:00.000000000 [3] 0.9396945134545455 1.067834674380165 43
2012-01-11T00:00:00.000000000 [2] 0.9754565886792453 1.081573205020921 39
2012-01-12T00:00:00.000000000 [1] 0.9851711026615969 1.079583333333333 27
2012-01-17T00:00:00.000000000 [3] 0.9582741066666667 1.361757941052631 77
2012-01-18T00:00:00.000000000 [2] 0.9343143135714286 1.081024825619835 44
2012-01-20T00:00:00.000000000 [5] 0.9395571464285715 1.124256414529915 66
2012-01-23T00:00:00.000000000 [4] 0.9536231884057971 1.169777777777778 58
2012-01-24T00:00:00.000000000 [3] 0.9595985576642336 1.086487623140496 47
2012-01-25T00:00:00.000000000 [2] 0.971472

In [7]:
print(data.loc[1:3])

        date               symbol     exdate   last_date cp_flag  \
1 2012-01-03  SPXW 120106C1210000 2012-01-06  03/01/2012       C   
2 2012-01-03  SPXW 120106C1220000 2012-01-06  03/01/2012       C   
3 2012-01-03  SPXW 120106C1230000 2012-01-06  03/01/2012       C   

   strike_price  best_bid  best_offer  volume  open_interest  ...  maturity  \
1          1210      66.0        68.8       1              2  ...         3   
2          1220      56.1        58.8       1              4  ...         3   
3          1230      46.2        49.1       1             21  ...         3   

          Open         High          Low        Close    Adj Close  \
1  1258.859985  1284.619995  1258.859985  1277.060059  1277.060059   
2  1258.859985  1284.619995  1258.859985  1277.060059  1277.060059   
3  1258.859985  1284.619995  1258.859985  1277.060059  1277.060059   

       Volume moneyness  midpoint  year  
1  3943710000  1.055422     67.40  2012  
2  3943710000  1.046771     57.45  2012  
3  

In [ ]:
data_train = data[data['date']<='2021-12-06']
data_val = data[(data['date']<='2022-05-19')]
data_test = data[data['date']>'2022-05-19']

In [ ]:
# assume we smooth for one specific day... do it separate for puts and calls!!!! double loop
def smooth(input_data):
    for j in input_data['cp_flag'].unique():
        for i in input_data['date'].unique():
            data_sub = input_data[(input_data['date']==i) & (input_data['cp_flag']==j)].copy()

            m = len(data_sub)
            s = m - np.sqrt(2*m)
            print(len(data_sub))

            if len(data_sub) < 16: # condition in order to be able to smooth in the first place: length should > (kx+1)*(ky+1) 
                print(i, j)
                continue
            # The smoother does not handle 1D data well.. in that case, do not smooth.
            if len(data_sub['maturity'].unique())==1:
                continue
            smoother = SmoothBivariateSpline(data_sub['maturity'], data_sub['moneyness'], data_sub['impl_volatility'], s=s)

            #print(smoother(data_sub['maturity'], data_sub['moneyness'], grid=False))
            
            input_data.loc[(input_data['date']==i) & (input_data['cp_flag']==j), 'IV_smooth'] = smoother(data_sub['maturity'], data_sub['moneyness'], grid=False)
    
    input_data['IV_smooth'] = input_data['IV_smooth'].fillna(input_data['impl_volatility'])
    return input_data

train_sample = smooth(data[data['date']<='2021-12-06']).copy()
train_val_sample = smooth(data[(data['date']<='2022-05-19')]).copy()

23
17
12
2012-01-05T00:00:00.000000000 C
27
20
16
21
17
41
16
28
25
18
25
8
2012-01-26T00:00:00.000000000 C
35
19
20
13
2012-02-01T00:00:00.000000000 C
16
29
18
33
14
2012-02-08T00:00:00.000000000 C
16
18
12
2012-02-14T00:00:00.000000000 C
14
2012-02-15T00:00:00.000000000 C
27
15
2012-02-21T00:00:00.000000000 C
12
2012-02-22T00:00:00.000000000 C
15
2012-02-23T00:00:00.000000000 C
25
27
21
20
9
2012-03-01T00:00:00.000000000 C
24
18
17
17
22
25
46
24
26
23
15
2012-03-20T00:00:00.000000000 C
15
2012-03-21T00:00:00.000000000 C
13
2012-03-22T00:00:00.000000000 C
26
24
24
18
46


C:\Users\hydra\AppData\Local\Temp\ipykernel_16644\1787010950.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data.loc[(input_data['date']==i) & (input_data['cp_flag']==j), 'IV_smooth'] = smoother(data_sub['maturity'], data_sub['moneyness'], grid=False)


18
15
2012-04-02T00:00:00.000000000 C
10
2012-04-03T00:00:00.000000000 C
12
2012-04-04T00:00:00.000000000 C
19
19
14
2012-04-11T00:00:00.000000000 C
16
31
25
29
22
23
20
15
2012-04-25T00:00:00.000000000 C
12
2012-04-26T00:00:00.000000000 C
23
18
15
2012-05-01T00:00:00.000000000 C
13
2012-05-02T00:00:00.000000000 C
13
2012-05-03T00:00:00.000000000 C
21
15
2012-05-07T00:00:00.000000000 C
19
22
8
2012-05-10T00:00:00.000000000 C
29
23
22
27
22
18
18
10
2012-05-24T00:00:00.000000000 C
22
18
21
13
2012-05-31T00:00:00.000000000 C
24
24
20
11
2012-06-06T00:00:00.000000000 C
20
23
19
16
24
22
22
11
2012-06-20T00:00:00.000000000 C
11
2012-06-21T00:00:00.000000000 C
26
20
24
15
2012-06-27T00:00:00.000000000 C
16
19
16
12
2012-07-03T00:00:00.000000000 C
20
22
25
19
20
15
2012-07-12T00:00:00.000000000 C
23
18
16
21
18
16
13
2012-07-25T00:00:00.000000000 C
14
2012-07-26T00:00:00.000000000 C
27
25
25
21
12
2012-08-02T00:00:00.000000000 C
25
21
16
12
2012-08-08T00:00:00.000000000 C
8
2012-08-09T00:00:

C:\Users\hydra\AppData\Local\Temp\ipykernel_16644\1787010950.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['IV_smooth'] = input_data['IV_smooth'].fillna(input_data['impl_volatility'])
C:\Users\hydra\AppData\Local\Temp\ipykernel_16644\1787010950.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data.loc[(input_data['date']==i) & (input_data['cp_flag']==j), 'IV_smooth'] = smoother(data_sub['maturity'], data_sub['moneyness'], grid=False)


186
207
204
285
346
266
261
319
221
244
173
170
147
249
171
274
238
210
113
188
191
206
225
130
195
232
184
321
295
284
184
332
442
335
315
285
295
291
267
232
235
247
196
186
176
216
380
311
264
315
217
223
228
306
257
314
335
293
261
210
256
285
277
269
263
398
409
352
283
156
197
245
229
251
256
234
215
230
205
238
245
148
145
225
371
229
250
250
169
112
176
225
241
219
291
343
329
351
312
324
401
323
380
357
363
395
353
375
429
523
437
367
482
401
427
430
347
642
598
541
576
536
386
387
306
297
207
386
292
314
277
327
339
205
355
273
324
322
309
295
363
598
650
573
455
314
523
456
644
397
541
486
319
408
346
356
439
369
380
347
668
507
728
522
304
342
541
371
534
553
469
390
488
418
416
352
482
467
286
693
716
687
667
446
470
414
388
400
506
632
541
408
480
283
349
373
386
380
606
618
629
413
401
197
368
407
384
353
428
474
364
582
492
579
507
437
464
517
446
396
402
512
584
940
943
850
656
392


C:\Users\hydra\AppData\Local\Temp\ipykernel_16644\1787010950.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['IV_smooth'] = input_data['IV_smooth'].fillna(input_data['impl_volatility'])


In [34]:
train_sample.to_csv('data/final/smoothed/data_train.csv')
train_val_sample.to_csv('data/final/smoothed/data_train_val.csv')

In [35]:
data_val = data[(data['date']>'2021-12-06') & (data['date']<='2022-05-19')]
data_test = data[data['date']>'2022-05-19']

In [36]:
data_val.to_csv('data/final/evaluation/validation_set.csv')
data_test.to_csv('data/final/evaluation/test_set.csv')

In [37]:
# Do the same for the long maturity set

long_data = pd.read_csv('data/before_pre/data_adj_long.csv')

In [41]:
n = len(long_data)
print(n)
index_70th = int(0.7 * n) - 1  # Subtract 1 because indices are zero-based
index_80th = int(0.8 * n) - 1

# Retrieve the corresponding dates
date_70th = long_data.loc[index_70th, 'date']
date_80th = long_data.loc[index_80th, 'date']
print(f"Date at 70th percentile: {date_70th}")
print(f"Date at 80th percentile: {date_80th}")

8102121
Date at 70th percentile: 2021-08-09
Date at 80th percentile: 2022-02-24


In [46]:
# 70 percent train, 10 percent validation, and 20 percent test
data_val_long = long_data[(data['date']>date_70th) & (long_data['date']<=date_80th)]
data_test_long = long_data[long_data['date']>date_80th]
data_val_long.to_csv('data/final/evaluation/validation_set_long.csv')
data_test_long.to_csv('data/final/evaluation/test_set_long.csv')

In [48]:
print(long_data.columns)

Index(['date', 'symbol', 'exdate', 'last_date', 'cp_flag', 'strike_price',
       'best_bid', 'best_offer', 'volume', 'open_interest', 'impl_volatility',
       'delta', 'gamma', 'vega', 'theta', 'optionid', 'am_settlement',
       'expiry_indicator', 'maturity', 'Open', 'High', 'Low', 'Close',
       'Adj Close', 'Volume', 'moneyness', 'midpoint', 'year'],
      dtype='object')


In [ ]:
train_sample_long = smooth(long_data[long_data['date']<=date_70th]).copy()
train_val_sample_long = smooth(long_data[long_data['date']<=date_80th]).copy()

In [ ]:
train_sample_long.to_csv('data/final/smoothed/data_train_long.csv')
train_val_sample_long.to_csv('data/final/smoothed/data_train_val_long.csv')